In [ ]:
import torchaudio
import torch
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Audio, display
from tqdm import tqdm

from src.configs import StreamingTaskConfig


noise_path_1 = 'speech_commands/_background_noise_/exercise_bike.wav'
noise_path_2 = 'speech_commands/_background_noise_/running_tap.wav'
command_path = "speech_commands/sheila/dc269564_nohash_1.wav"
model_path = "models/baseline_streaming_share_hs.pt"
sample_rate = 16000

noise_1 = torchaudio.load(noise_path_1)[0].squeeze()
noise_2 = torchaudio.load(noise_path_2)[0].squeeze()
command = torchaudio.load(command_path)[0].squeeze()

model = torch.jit.load(model_path)
concat_wav = torch.cat([noise_1[-160000:], command, noise_2[160000:320000]])
display(Audio(concat_wav, rate=16000))

predictions = []

for frame in tqdm(concat_wav):
    predictions.append(model(frame)[1].item())

sns.lineplot(np.arange(len(predictions)) / sample_rate, predictions)
plt.xlabel("Time (s)")
plt.ylabel("Probability of command")
plt.title("Probability of commmand 'Sheila' over time")

In [ ]:
model_path_2 = "models/baseline_streaming_not_share_hs.pt"
model_2 = torch.jit.load(model_path)

predictions = []

for frame in tqdm(concat_wav):
    predictions.append(model(frame)[1].item())

sns.lineplot(np.arange(len(predictions)) / sample_rate, predictions)
plt.xlabel("Time (s)")
plt.ylabel("Probability of command")
plt.title("Probability of commmand 'Sheila' over time (no sharing of hidden states")